## Рекомендательные системы

In [3]:
"""
полезные ссылки:
    
https://www.kaggle.com/kanncaa1/recommendation-systems-tutorial
https://blog.cambridgespark.com/tutorial-practical-introduction-to-recommender-systems-dbe22848392b
https://datasciencebeginners.com/2018/10/31/ultimate-guide-on-how-to-build-recommender-systems-with-case-study/
https://habr.com/ru/company/yandex/blog/241455/
https://habr.com/ru/company/yandex/blog/441586/
""";

В этом ноутбуке мы применим алгоритм коллаборативной фильтрации на item-base подходе. Работать мы будем с датасетом MovieLens, который содержит в себе информацию об оценках фильмов пользователями одноименного сайта.

Давайте загрузим необходимые библиотеки.

In [78]:
import zipfile
from collections import defaultdict, Counter
import datetime
import pandas as pd

from scipy import linalg
import scipy.sparse as sps
import numpy as np
import matplotlib.pyplot as plt

Скачаем данные

In [2]:
!wget http://files.grouplens.org/datasets/movielens/ml-1m.zip

--2020-06-02 23:53:59--  http://files.grouplens.org/datasets/movielens/ml-1m.zip
Resolving files.grouplens.org (files.grouplens.org)... 128.101.65.152
Connecting to files.grouplens.org (files.grouplens.org)|128.101.65.152|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5917549 (5.6M) [application/zip]
Saving to: <<ml-1m.zip.2>>

ml-1m.zip.2         100%[===================>]   5.64M  2.47MB/s    in 2.3s    

2020-06-02 23:54:01 (2.47 MB/s) - <<ml-1m.zip.2>> saved [5917549/5917549]



Распакуем данные и посмотрим, как они устроены.

In [2]:
with zipfile.ZipFile("ml-1m.zip", "r") as z:
    print("files in archive")
    print(z.namelist())
    print("movies")
    with z.open("ml-1m/movies.dat") as m:
        print(str(m.readline()).split("::"))
    print("users")
    with z.open("ml-1m/users.dat") as m:
        print(str(m.readline()).split("::"))
    print("ratings")
    with z.open("ml-1m/ratings.dat") as m:
        print(str(m.readline()).split("::"))

files in archive
['ml-1m/', 'ml-1m/movies.dat', 'ml-1m/ratings.dat', 'ml-1m/README', 'ml-1m/users.dat']
movies
['b"1', 'Toy Story (1995)', 'Animation|Children\'s|Comedy\\n"']
users
["b'1", 'F', '1', '10', "48067\\n'"]
ratings
["b'1", '1193', '5', "978300760\\n'"]


Мы видим, что в архиве лежит информация о фильмах. Это movieId фильма, название и жанр. О пользователях нам известен userId, пол (F, M), возраст, закодированная информация о трудоуствройстве и zip-code. И информация о рейтинге: userId, movieId, оценка и момент времени, когда оценка была сделана. Давайте прочитаем данные.

In [3]:
# read data
movies = {} # id
users = {} # id
ratings = defaultdict(list) # user-id

with zipfile.ZipFile("ml-1m.zip", "r") as z:
    # parse movies
    with z.open("ml-1m/movies.dat") as m:
        for line in m:
            MovieID, Title, Genres = line.decode('iso-8859-1').strip().split("::")
            MovieID = int(MovieID)
            Genres = Genres.split("|")
            movies[MovieID] = {"Title": Title, "Genres": Genres}
    
    # parse users
    with z.open("ml-1m/users.dat") as m:
        fields = ["UserID", "Gender", "Age", "Occupation", "Zip-code"]
        for line in m:
            row = list(zip(fields, line.decode('iso-8859-1').strip().split("::")))
            data = dict(row[1:])
            data["Occupation"] = int(data["Occupation"])
            users[int(row[0][1])] = data
    
    # parse ratings
    with z.open("ml-1m/ratings.dat") as m:
        for line in m:
            UserID, MovieID, Rating, Timestamp = line.decode('iso-8859-1').strip().split("::")
            UserID = int(UserID)
            MovieID = int(MovieID)
            Rating = int(Rating)
            Timestamp = int(Timestamp)
            ratings[UserID].append((MovieID, Rating, datetime.datetime.fromtimestamp(Timestamp)))

Посмотрим на данные

In [5]:
print(users[3])

{'Gender': 'M', 'Age': '25', 'Occupation': 15, 'Zip-code': '55117'}


In [6]:
print(ratings[4][2])

(2951, 4, datetime.datetime(2000, 12, 31, 23, 24, 42))


In [7]:
len(ratings)

6040

In [9]:
a = ratings.values()

In [10]:
a = list(a)

In [12]:
a[0][:20]

[(1193, 5, datetime.datetime(2001, 1, 1, 1, 12, 40)),
 (661, 3, datetime.datetime(2001, 1, 1, 1, 35, 9)),
 (914, 3, datetime.datetime(2001, 1, 1, 1, 32, 48)),
 (3408, 4, datetime.datetime(2001, 1, 1, 1, 4, 35)),
 (2355, 5, datetime.datetime(2001, 1, 7, 2, 38, 11)),
 (1197, 3, datetime.datetime(2001, 1, 1, 1, 37, 48)),
 (1287, 5, datetime.datetime(2001, 1, 1, 1, 33, 59)),
 (2804, 5, datetime.datetime(2001, 1, 1, 1, 11, 59)),
 (594, 4, datetime.datetime(2001, 1, 1, 1, 37, 48)),
 (919, 4, datetime.datetime(2001, 1, 1, 1, 22, 48)),
 (595, 5, datetime.datetime(2001, 1, 7, 2, 37, 48)),
 (938, 4, datetime.datetime(2001, 1, 1, 1, 29, 12)),
 (2398, 4, datetime.datetime(2001, 1, 1, 1, 38, 1)),
 (2918, 4, datetime.datetime(2001, 1, 1, 1, 35, 24)),
 (1035, 5, datetime.datetime(2001, 1, 1, 1, 29, 13)),
 (2791, 4, datetime.datetime(2001, 1, 1, 1, 36, 28)),
 (2687, 3, datetime.datetime(2001, 1, 7, 2, 37, 48)),
 (2018, 4, datetime.datetime(2001, 1, 1, 1, 29, 37)),
 (3105, 5, datetime.datetime(2001, 1,

Теперь разобьем данные на тренировочную и тестовую выборку. Разбивать будем рейтинги по времени. Для начала найдем дату на которую было выставленно 80% рейтингов в датасете. И все рейтинги, что были до, попадут в train, а остальные в test.

In [13]:
times = []
for user_ratings in ratings.values():
    times.extend([x[2] for x in user_ratings])

In [14]:
times = sorted(times)

In [15]:
threshold_time = times[int(0.8 * len(times))]

In [16]:
train = []
test = []
for user_id, user_ratings in ratings.items():
    train.extend((user_id, rating[0], rating[1] / 5.0) for rating in user_ratings if rating[2] <= threshold_time)
    test.extend((user_id, rating[0], rating[1] / 5.0) for rating in user_ratings if rating[2] > threshold_time)
print("ratings in train:", len(train))
print("ratings in test:", len(test))

ratings in train: 800168
ratings in test: 200041


In [79]:
train_df = pd.DataFrame(train, columns=['uid', 'iid', 'rating'])

In [86]:
pivot_table = train_df.pivot_table(index = ["uid"],columns = ["iid"],values = "rating")
pivot_table.head(10)

iid,1,2,3,4,5,6,7,8,9,10,...,3943,3944,3945,3946,3947,3948,3949,3950,3951,3952
uid,,,,,,,,,,,,,,,,,,,,,
635,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,0.8,NaN,NaN,NaN,NaN
636,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
637,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
638,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
639,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
640,NaN,NaN,NaN,NaN,NaN,0.8,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
641,NaN,0.8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
642,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
643,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [87]:
pivot_table.shape

(5400, 3662)

In [111]:
movie_watched = pivot_table[2]
similarity_with_other_movies = pivot_table.corrwith(movie_watched)
similarity_with_other_movies = similarity_with_other_movies.sort_values(ascending=False)
similarity_with_other_movies.head(100)

iid
1421    1.000000
148     1.000000
2342    1.000000
3855    1.000000
1657    1.000000
          ...   
3913    0.866025
3570    0.866025
626     0.859338
846     0.849536
2131    0.845906
Length: 100, dtype: float64

In [136]:
user_id = pivot_table.loc[651]
similarity_with_other_users = pivot_table.corrwith(user_id, axis=1)
similarity_with_other_users = similarity_with_other_users.sort_values(ascending=False)
similarity_with_other_users.head(100)

uid
3264    1.000000
4421    1.000000
4151    1.000000
874     1.000000
651     1.000000
          ...   
2640    0.807692
3124    0.805659
1956    0.804783
1371    0.804030
1907    0.802955
Length: 100, dtype: float64

In [127]:
def get_user_sim(uid_1, uid_2):
    user_id = pivot_table.loc[uid_1]
    similarity_with_other_users = pivot_table.corrwith(user_id, axis=1)
    similarity_with_other_users = similarity_with_other_users.sort_values(ascending=False)
    corr = similarity_with_other_users[uid_2]
    if np.isnan(corr):
        return 0
    else:
        return corr

In [138]:
get_user_sim(651, 1907)

0.8029550685469663

In [169]:
def get_rat(uid, iid):
    u_mean = pivot_table.loc[uid].mean()
    user_id = pivot_table.loc[uid]
    similarity_with_other_users = pivot_table.corrwith(user_id, axis=1)
    add = (((pivot_table[iid] - pivot_table.mean(axis=1)) * 
            similarity_with_other_users).sum() / similarity_with_other_users.sum())
    return u_mean + add 

In [174]:
get_rat(639, 7)

0.7416224362684901

In [177]:
test_rats = test_by_user[639][:10]
test_rats

[(2051, 0.6),
 (1250, 0.6),
 (3793, 0.8),
 (2991, 0.8),
 (2054, 0.8),
 (2055, 0.8),
 (1253, 0.8),
 (2993, 0.6),
 (2056, 0.8),
 (2057, 0.6)]

In [180]:
pred_rats = [get_rat(639, i[0]) for i in test_rats]

In [181]:
pred_rats

[0.733831464383334,
 0.7597760358147008,
 0.7514682720382363,
 0.7365093043923043,
 0.7182065391333685,
 0.7379815462849528,
 0.7442018565564176,
 0.7379248436332352,
 0.7394989714491438,
 0.7395262101939952]

## ALS факторизация

В этом подходе оценка $r_{ui}$ пользователя $u$, поставленная фильму $i$, ищется как скалярное произведение векторов $p_u$ и $q_i$ в некотором пространстве $\mathbb{R}^K$ латентных признаков:

$$
	\hat{r}_{ui} = p_u^T q_i 
$$


Иными словами, модель находит пространство признаков, в котором мы описываем и фильмы и пользователей и в котором рейтинг является мерой близости между фильмами и пользователями.
	
Для настройки модели будем минимизировать следующую ошибку:
	$$
	\sum_{(u, i, r_{ui})} (r_{ui} - p_u^T q_i)^2 + \lambda_{p} p_u^T p_u + \lambda_{q} q_i^T q_i,
	$$
	где суммирование ведется по всем тройкам $(u, i, r_{ui})$ выборки, слагаемые с $\lambda_{p}$ и $\lambda_{q}$ добавлены для регуляризации.


In [61]:
LATENT_SIZE = 10

# регуляризаторы
lambda_p = 0.2
lambda_q = 0.001



Рассмотрим работу алгоритма ALS, продолжая работать с данными Movielens.

Посчитаем количество пользователей и фильмов

In [62]:
n_users = max([e[0] for e in train]) + 1
n_items = max([e[1] for e in train]) + 1

Инициализируем латентные представления для пользователей

In [63]:
p = 0.1 * np.random.random((n_users, LATENT_SIZE))
q = 0.1 * np.random.random((n_items, LATENT_SIZE))

Составим словарь взаимодействий по фильму

In [64]:
train_by_item = defaultdict(list)
for u, i, r in train:
    train_by_item[i].append((u, r))

Теперь составим матрицу $P$ из векторов $p_u$ и матрицу $Q$ из векторов $q_i$. Матрицей $Q[u] \in \mathbb{R}^{n_u \times K}$ будем обозначать подматрицу матрицы $Q$ только для товаров, оцененных пользователем $u$, где $n_u$ - количество оценок пользователя $u$.
	
Шаг перенастройки $p_u$ при фиксированной матрице $Q$ сводится к настройке ridge-регрессии и выглядит так:
	$$
	A_u = Q[u]^T Q[u] \\
	d_u = Q[u]^T r_u \\
	p_u = (\lambda_p n_u I + A_u)^{-1} d_u
	$$

In [65]:
def compute_p(p, q, train_by_user):
    for u, rated in train_by_user.items():
        rated_items = [i for i, _ in rated]
        rated_scores = np.array([r for _, r in rated])
        Q = q[rated_items, :]
        A = (Q.T).dot(Q)
        d = (Q.T).dot(rated_scores)
        p[u, :] = np.linalg.solve(lambda_p * len(rated_items) * np.eye(LATENT_SIZE) + A, d)
    return p

p = compute_p(p, q, train_by_user)

In [66]:
p.shape

(6041, 10)


Формулы для обновления $q_i$ при фиксированной матрице $P$ выглядят аналогично, реализация будет выглядеть следующим образом:


In [67]:
def compute_q(p, q, train_by_item):
    for i, rated in train_by_item.items():
        rated_users = [j for j, _ in rated]
        rated_scores = np.array([s for _, s in rated])
        P = p[rated_users, :]
        A = (P.T).dot(P)
        d = (P.T).dot(rated_scores)
        q[i, :] = np.linalg.solve(lambda_q * len(rated_users) * np.eye(LATENT_SIZE) + A, d)
    return q

q = compute_q(p, q, train_by_item)

In [68]:
q.shape

(3953, 10)

Теперь мы можем сделать предсказание всей матрицы оценок

In [69]:
predictions = p.dot(q.T)

Качество работы модели будем определять по среднеквадратичной ошибке между рейтингом и предсказанием

In [70]:
def train_error(predictions):
    return np.mean([(predictions[u, i] - r) ** 2 for u, i, r in train])

def test_error(predictions):
    return np.mean([(predictions[u, i] - r) ** 2 for u, i, r in test])

Теперь полностью реализуем метод: в ALS проводятся $N$ итераций, в рамках каждой сначала оптимизируется $p$ при фиксированном $q$, затем $q$ при фиксированном $p$.

In [71]:
%%time
N_ITER = 20
for iter in range(N_ITER):
    p = compute_p(p, q, train_by_user)
    q = compute_q(p, q, train_by_item)

    predictions = p.dot(q.T)
    
    print(iter, train_error(predictions), test_error(predictions))

0 0.030251476868687323 0.14941125813178488
1 0.026844134262376335 0.14170226656558305
2 0.025675405357217174 0.135024831103676
3 0.025225989788600606 0.1290203135558301
4 0.02500526806547699 0.12364174601310443
5 0.024866665397898717 0.11881926374585368
6 0.02476756458057769 0.11449019490396228
7 0.02469447512070576 0.11059383423802333
8 0.024641363006368013 0.10707722458686651
9 0.02460345135248366 0.10389590514076302
10 0.024576633832460882 0.10101261333825387
11 0.024557742960241134 0.0983957134820434
12 0.024544508609784795 0.0960177922394579
13 0.02453534084169045 0.09385465032313375
14 0.024529139344836366 0.09188486804780116
15 0.0245251301286065 0.09008961431031255
16 0.0245227526679149 0.08845229265497416
17 0.024521592709060343 0.0869581087544063
18 0.024521338298668094 0.08559376535113966
19 0.024521752865116455 0.0843472764342248
CPU times: user 55.4 s, sys: 1.91 s, total: 57.3 s
Wall time: 34.8 s


In [72]:
p.shape, q.shape

((6041, 10), (3953, 10))